This notebook builds on `Preprocessing.ipynb' and uses the data to construct surrogate models that can be used for controller optimization studies.



#  Surrogate Model for Controller Gains Optimization

Models that balance accuracy against computational costs are advantageous when designing floating offshore wind turbines (FOWT) using OpenFAST as the high-fidelity model through optimization studies, as several hundred predictive function evaluations might be necessary to identify the optimal solution. 

OpenFAST models the dynamic response of FOWT systems as a system of differential-algebraic equations (DAE).
Whenever OpenFAST is evaluated for a load case, this DAE system is solved to obtain the dynamic response of the FOWT. OpenFAST also uses different modules like AeroDyn, Hydrodyn etc. to capture the appropriate physics.
Therefore evaluating OpenFAST can be computationally expensive, making it unsuitable for optimization based studies where several hundred model evaluations might be required.

## Surrogate Model Requirements

Consider the following figure illustrating the steps involved in a controller optimization study:

<img src="EAB-copt.svg" width="600"/>

The goal is to develop a surrogate model to replace the aero-hydro-servo-elastic model that can predict the timeseries of outputs given the inputs and controls. Once constructed, this model can then be coupled with the controller as shown in the figure:

<img src="SM2.svg" width="800"/>


There are different approaches that can be used to construct such a surrogate model such as classic system identification (sys-id) methods and long-short-term memory (LSTM) networks.
These methods can be classified as 'black-box' approaches to map the inputs to the outputs. They have been widely utilized for various complex engineering systems.

However, in this example we use the derivative function surrogate modeling (DFSM) approach to construct the surrogate model.

## Derivative Function Surrogate Model (DFSM)

DFSM are surrogate models of the dynamic function or derivative function, that describes how the state derivatives evolve for the given system.
A key assumption that is made when using DFSM approach is that the system dynamics can be described by an ordinary differential equation (ODE) of the following form:
    \begin{align}
            \dot{\xi} &= f(\xi,u)\\
            y &= g(\xi,u)
    \end{align}
Where $\xi$ denotes the states, $u$ denotes the controls, $y$ denotes the outputs, and $f$, $g$ correspond to the state derivative and output functions respectively.      

DFSM approximates $f$ and $g$ as:
    \begin{align}
        \dot{\xi} &\approx f_{\text{DFSM}}(\xi,u)\\
        y &\approx g_{\text{DFSM}}(\xi,u) \\
    \end{align}

Unlike the aforementioned black-box approaches, the DFSM approach allows the user to prescribe a relation between the quantities approximated. In this study, we assume that the underlying ODE can be approximated as an linear-parameter varying (LPV) system, where the key parameter is the wind speed ($w$):
    \begin{align}
        f_{\text{DFSM}}(\xi,u) = A(w)\xi + B(w)u\\
        g_{\text{DFSM}}(\xi,u) = C(w)\xi + D(w)u \\
    \end{align}


## Model Construction

### Inputs
The inputs when constructing the DFSM are turbulent OpenFAST simulations for different wind speeds between $w \in [3,25]$. More details regarding these inputs, and how they are preprocessed can be found in `Preprocessing.ipynb'.

Once the inputs are available and preprocessed, then the LPV model is constructed as follows:
1. Individual state-space matrices $\Sigma = [A,B,C,D]$ are obtained for different wind speeds values for $w \in [3,25]$
2. The system matrices $\Sigma$ are then linearly interpolated over $w$ to obtain a continuous model. For more details regarding LPV modeling, please refer to [https://doi.org/10.1115/1.4063969](https://doi.org/10.1115/1.4063969) 

### Obtaining linear models
We solve an optimization problem to obtain the system matrices $\Sigma$. The $A$ and $B$ matrices are evaluated as part of one problem, and the $C$ and $D$ matrices are evaluate separately, as they predict different quantities, and have different requirements.


The problem solved to identify the A and B matrices can be formulated as:
\begin{align}
    \min_{\theta} \frac{Tr(E'*E)}{N} \\
    \text{s.t} \quad max(real(eig(A(\theta))) < 0\\
    \text{where:} \quad E = \dot{\xi}_{\text{act}} - \dot{\xi}_{\text{DFSM}}\\
                     \dot{\xi}_{\text{DFSM}} = A(\theta)\xi + B(\theta)u  \\
                     Tr~ \text{is the trace operation}
\end{align}
The constraint on the eigen value is required to ensure the the simulations and the state-space system remain stable.
We use a hybrid optimization strategy to solve this problem. A genetic algorithm is used to get a good estimate of $\theta$, then a gradient-based optimizer using the interior-point algorithm is used to solve the problem.

Then for the next wind speed $w_2$, the $A$ and $B$ matrices evaluated for the previous wind speed $w_1$ are used as the starting point in the gradient-based optimizer.